# 笔记本 5: LangChain 集成

[LangChain](https://python.langchain.com/docs/get_started/introduction.html)是一个用于开发由语言模型驱动的应用程序的流行库。您可以使用 LangChain 和 LLM 构建各种有趣的应用程序，例如 [聊天机器人](https://github.com/intel-analytics/BigDL/blob/main/python/llm/example/langchain/transformers_int4/chat.py)、[文档问答](https://github.com/intel-analytics/BigDL/blob/main/python/llm/example/langchain/transformers_int4/docqa.py)、[语音助手](https://github.com/intel-analytics/BigDL/blob/main/python/llm/example/langchain/transformers_int4/voiceassistant.py)。BigDL-LLM 提供 LangChain 集成（即 LLM wrapper 和 embedding），您可以像使用 [LangChain 中的其他 LLM wrapper](https://python.langchain.com/docs/integrations/llms/) 一样使用它们。

本笔记本将介绍如何使用 langchain 与 BigDL-LLM 交互。

## 5.1 安装

首先，在准备好的环境中安装 BigDL-LLM。有关环境配置的最佳实践，请参阅本教程的 [第二章]()。

In [ ]:
!pip install bigdl-llm[all]

接下来安装 LangChain.

In [ ]:
!pip install -U langchain==0.0.248

> **注意**
> 
> 我们建议使用 `langchain==0.0.248`，这个版本在我们的教程中不会出现问题。

## 5.3 LLM Wrapper

BigDL-LLM 提供了 `TransformersLLM` 和 `TransformersPipelineLLM` ，它们实现了 LangChain 的 LLM wrapper 的标准接口。

您可以使用 `TransformerLLM.from_model_id` 从 huggingface model_id 或路径实例化 `TransformerLLM`。您可以在 `model_kwargs` 中以字典形式传入与模型生成相关的参数（如 `temperature`, `max_length`）。让我们以 [`vicuna-7b-v1.5`](https://huggingface.co/lmsys/vicuna-7b-v1.5) 模型为例实例化创建一个 `TransformerLLM`。


In [ ]:
from bigdl.llm.langchain.llms import TransformersLLM

llm = TransformersLLM.from_model_id(
        model_id="lmsys/vicuna-7b-v1.5",
        model_kwargs={"temperature": 0, "max_length": 1024, "trust_remote_code": True},
    )

> **注意**
>
> `TransformersPipelineLLM` 的实例化方式与 `TransformersLLM` 类似，都是通过 huggingface model_id 或路径、`model_kwargs` 和 `pipeline_kwargs` 来实现的。此外，还有一个额外的 `task` 参数用于指定要执行的任务类型。

使用 prompt 模板来格式化 prompt，然后调用 `llm` 测试生成的结果。

> **注意**
>
> `max_new_tokens` 参数定义了生成的 token 的最大数量。

In [197]:
prompt = "What is AI?"
VICUNA_PROMPT_TEMPLATE = "USER: {prompt}\nASSISTANT:"
result = llm(prompt=VICUNA_PROMPT_TEMPLATE.format(prompt=prompt), max_new_tokens=128)

AI stands for "Artificial Intelligence." It refers to the development of computer systems that can perform tasks that typically require human intelligence, such as visual perception, speech recognition, decision-making, and language translation. AI can be achieved through a combination of techniques such as machine learning, natural language processing, computer vision, and robotics. The ultimate goal of AI research is to create machines that can think and learn like humans, and can even exceed human capabilities in certain areas.


您也可以在 LLM 上使用 `generate` 来获取多组结果。

In [ ]:
llm_result = llm.generate([VICUNA_PROMPT_TEMPLATE.format(prompt="Tell me a joke"), VICUNA_PROMPT_TEMPLATE.format(prompt="Tell me a poem")]*3)

In [199]:
print("-"*20+"number of generations"+"-"*20)
print(len(llm_result.generations))
print("-"*20+"the first generation"+"-"*20)
print(llm_result.generations[0][0].text)

--------------------number of generations--------------------
6
--------------------the first generation--------------------
USER: Tell me a joke
ASSISTANT: Why did the tomato turn red?

Because it saw the salad dressing!


## 5.4 使用 Chains

现在，让我们开始使用 [Chains](https://docs.langchain.com/docs/components/chains/) 中的 LLM wrapper 和 embedding 。

>**注意**
>
> Chain 是 LangChain 中的一个重要组件，它将一系列模块化组件（甚至是其他 chain）组合在一起，以实现特定目的。chain 中的组件可以是 prompt 模板、模型、内存缓冲区等。

### 5.4.1 LLMChain

首先，我们尝试使用一个简单的 chain `LLMChain`. 

如下创建一个简单的 prompt 模板：

In [252]:
from langchain import PromptTemplate

template ="USER: {question}\nASSISTANT:"
prompt = PromptTemplate(template=template, input_variables=["question"])

现在，使用我们在上一节创建的 `llm` 和刚刚创建的 prompt 模板来实例化一个 `LLMChain` 。

In [201]:
from langchain import LLMChain

llm_chain = LLMChain(prompt=prompt, llm=llm)

现在，让我们向 llm 问一个问题，并在 `LLMChain` 上调用 `run` 来获取响应。

In [203]:
question = "What is AI?"
result = llm_chain.run(question)

AI stands for "Artificial Intelligence." It refers to the development of computer systems that can perform tasks that typically require human intelligence, such as visual perception, speech recognition, decision-making, and language translation. AI can be achieved through a combination of techniques such as machine learning, natural language processing, computer vision, and robotics. The ultimate goal of AI research is to create machines that can think and learn like humans, and can even exceed human capabilities in certain areas.


### 5.4.2 Conversation Chain

在构建聊天应用程序时，我们可以使用更复杂的 chain 和内存缓冲区来记忆聊天记录。这对于实现多轮聊天体验非常有用。

> **注意**
>
> `ConversationBufferMemory` 是 LangChain 中的一种存储器，用于存储对话中的信息并以不同格式提取出来。

In [2]:
from langchain import PromptTemplate
from langchain.chains import ConversationChain
from langchain.chains.conversation.memory import ConversationBufferMemory

template = "The following is a friendly conversation between a human and an AI.\
    \nCurrent conversation:\n{history}\nHuman: {input}\nAI Asistant:"
prompt = PromptTemplate(template=template, input_variables=["history", "input"])
conversation_chain = ConversationChain(
    verbose=True,
    prompt=prompt,
    llm=llm,
    memory=ConversationBufferMemory(),
    llm_kwargs={"max_new_tokens": 256},
)

In [3]:
query ="Good morning AI!" 
result = conversation_chain.run(query)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI.    
Current conversation:

Human: Good morning AI!
AI Asistant:
Good morning! How can I assist you today?

> Finished chain.


In [4]:
query ="Tell me about Intel." 
result = conversation_chain.run(query)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI.    
Current conversation:
Human: Good morning AI!
AI: The following is a friendly conversation between a human and an AI.    
Current conversation:

Human: Good morning AI!
AI Asistant: Good morning! How can I assist you today?
Human: Tell me about Intel.
AI Asistant:
Intel is a multinational technology company that specializes in the development and manufacturing of computer hardware and technology solutions. It was founded in 1976 by Robert Noyce and Gordon Moore, and is headquartered in Santa Clara, California. Intel is best known for its microprocessors, which are the "brains" of most computers and devices. They also produce other hardware such as motherboard chipsets, flash memory, and other components. Intel is one of the largest and most well-known technology companies in the world, and is a leader in the development of new technologies and inn

### 5.4.3 MathChain

让我们尝试通过使用 `MathChain` 来让 LLM 解决一些数学问题。

> **注意** 
> MathChain 通常需要使用较大的 `max_length` 来实例化 LLM，例如 1024


In [255]:
from langchain.chains import LLMMathChain

MATH_CHAIN_TEMPLATE ="Question: {question}\nAnswer:"
prompt = PromptTemplate(template=MATH_CHAIN_TEMPLATE, input_variables=["question"])
llm_math = LLMMathChain.from_llm(prompt=prompt, llm=llm, verbose=True)

In [256]:
question = "What is 13 raised to the 2 power"
llm_math.run(question)



> Entering new LLMMathChain chain...
What is 13 raised to the 2 power

13 raised to the 2 power is equal to 13 \* 13, which is 169.
Question: What is 13 raised to the 2 power
Answer: 13 raised to the 2 power is equal to 13 \* 13, which is 169.
> Finished chain.


'Answer:  13 raised to the 2 power is equal to 13 \\* 13, which is 169.'

### 5.5 通过文档回答问题

如果您有一些文本文档（PDF、博客、Notion 页面等），并想要询问与这些文档内容相关的问题。鉴于 LLM 在理解文本方面的熟练程度，它是做这方面工作的一个很好用的工具。

#### 5.5.1 安装

请在运行示例前安装必要的依赖库。

In [ ]:
!pip install -U faiss-cpu

#### 5.5.1 加载文档

为方便起见，这里我们使用文本字符串作为加载文档。非结构化的数据可以从许多来源加载。请访问 [LangChain integration hub](https://integrations.langchain.com/) 来浏览全部加载器。

In [76]:
input_doc = "\
    BigDL: fast, distributed, secure AI for Big Data\n\n\
    BigDL seamlessly scales your data analytics & AI applications from laptop to cloud, with the following libraries:\
        Orca: Distributed Big Data & AI (TF & PyTorch) Pipeline on Spark and Ray\
        Nano: Transparent Acceleration of Tensorflow & PyTorch Programs on XPU\
        DLlib: “Equivalent of Spark MLlib” for Deep Learning\
        Chronos: Scalable Time Series Analysis using AutoML\
        Friesian: End-to-End Recommendation Systems\
        PPML: Secure Big Data and AI (with SGX Hardware Security)\
        LLM: A library for running large language models with very low latency using low-precision techniques on Intel platforms\n\n\
    "

#### 5.5.2 拆分输入文件的文本

[Text splitters](https://python.langchain.com/docs/modules/data_connection/document_transformers/) 将文档分割成指定大小的片段。在这里，我们将文档分割成块，用于嵌入和向量存储。

> **注意**
> 
> `CharacterTextSplitter` 只对分隔符（默认为 `'\n\n'`）进行分割。
>
> `chunk_size` 是最大的分割字符数，前提是如果可以分割的话。
>
> `chunk_overlap` 是每次分割之间的重叠字符数。

In [ ]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=650, chunk_overlap=0)
texts = text_splitter.split_text(input_doc)

#### 5.5.3 创建嵌入并存储到向量存储中

拆分文档后，我们需要存储拆分内容，以便日后根据输入查询进行搜索。最常见的方法是嵌入每个分片的内容，然后将嵌入向量存储在向量存储中。

众所周知，在 Transformers 中，有一些嵌入层可以将非结构化数据转换为嵌入向量（一个由数字组成的数组），从而对其执行各种操作。嵌入向量代表现实世界中的对象和概念，如单词、文档等。

BigDL-LLM 提供了 `TransformersEmbeddings`，它允许你使用 LLM 从文本输入中获取嵌入。

TransformersEmbeddings "的实例化方法与 "TransformersLLM "类似

In [ ]:
from bigdl.llm.langchain.embeddings import TransformersEmbeddings

embeddings = TransformersEmbeddings.from_model_id(model_id="lmsys/vicuna-7b-v1.5")

介绍完 `TransformersEmbeddings` 后，让我们来创建嵌入并存储到向量存储中。向量存储负责存储嵌入数据并执行向量搜索。这里我们以 [Faiss](https://faiss.ai/index.html)为例，Faiss 是一个用于对密集向量进行高效相似性搜索和聚类的库。

In [82]:
from langchain.vectorstores import FAISS

docsearch = FAISS.from_texts(texts, embeddings, metadatas=[{"source": str(i)} for i in range(len(texts))]).as_retriever()

#### 5.5.4 获取相关文档

如前所述，嵌入向量可以用于表示查询和文档。这种表示法可以将语义相似性转化为向量空间中的接近性。因此，我们可以通过这种相似性来搜索文档。

In [83]:
query = "What is BigDL?"
docs = docsearch.get_relevant_documents(query)
print("-"*20+"number of relevant documents"+"-"*20)
print(len(docs))

--------------------number of relevant documents--------------------
2


#### 5.5.5 准备 chain

In [84]:
from langchain.chains.chat_vector_db.prompts import QA_PROMPT
from langchain.chains.question_answering import load_qa_chain

doc_chain = load_qa_chain(
    llm, chain_type="stuff", prompt=QA_PROMPT
)

#### 5.5.6 生成

In [85]:
result = doc_chain.run(input_documents=docs, question=query)

BigDL is a fast, distributed, and secure AI library for Big Data. It enables seamless scaling of data analytics and AI applications from laptops to the cloud. BigDL supports various libraries, including Orca, Nano, DLlib, Chronos, Friesian, PPML, and LLM. These libraries cater to different use cases, such as distributed Big Data processing, transparent acceleration of TensorFlow and PyTorch programs, scalable time series analysis, end-to-end recommendation systems, and secure Big Data and AI with SGX hardware security. BigDL aims to provide a unified platform for AI and data analytics, making it easier for developers to build and deploy their applications at scale.
